In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
# Hyper-parameters 
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\train-images-idx3-ubyte.gz to ../../data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



...

Extracting ../../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\MNIST\raw

Processing...


D:\anaconda\envs\python36\lib\site-packages\torchvision\datasets\mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [4]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


In [6]:
model=nn.Linear(input_size,num_classes)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

total_step=len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=images.reshape(-1,input_size)
        
        outputs=model(images)
        loss=criterion(outputs,labels)
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 2.2510
Epoch [1/5], Step [200/600], Loss: 2.1188
Epoch [1/5], Step [300/600], Loss: 2.0357
Epoch [1/5], Step [400/600], Loss: 2.0021
Epoch [1/5], Step [500/600], Loss: 1.8636
Epoch [1/5], Step [600/600], Loss: 1.7938
Epoch [2/5], Step [100/600], Loss: 1.6909
Epoch [2/5], Step [200/600], Loss: 1.5913
Epoch [2/5], Step [300/600], Loss: 1.6433
Epoch [2/5], Step [400/600], Loss: 1.6161
Epoch [2/5], Step [500/600], Loss: 1.5236
Epoch [2/5], Step [600/600], Loss: 1.4015
Epoch [3/5], Step [100/600], Loss: 1.3799
Epoch [3/5], Step [200/600], Loss: 1.3607
Epoch [3/5], Step [300/600], Loss: 1.3519
Epoch [3/5], Step [400/600], Loss: 1.3427
Epoch [3/5], Step [500/600], Loss: 1.2590
Epoch [3/5], Step [600/600], Loss: 1.2708
Epoch [4/5], Step [100/600], Loss: 1.2402
Epoch [4/5], Step [200/600], Loss: 1.2436
Epoch [4/5], Step [300/600], Loss: 1.2321
Epoch [4/5], Step [400/600], Loss: 1.1473
Epoch [4/5], Step [500/600], Loss: 1.0894
Epoch [4/5], Step [600/600], Loss:

In [7]:
with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.reshape(-1,input_size) #一行为一样本
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum()
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


Accuracy of the model on the 10000 test images: 83.13999938964844 %


## Logistic 

In [21]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [18]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [19]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [22]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(hidden_size,num_classes)

    def forward(self,x):
        out=self.fc1(x)
        out=self.relu(out)
        out=self.fc2(out)
        return out 
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

total_step=len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.2882
Epoch [1/5], Step [200/600], Loss: 0.3217
Epoch [1/5], Step [300/600], Loss: 0.2654
Epoch [1/5], Step [400/600], Loss: 0.1442
Epoch [1/5], Step [500/600], Loss: 0.2622
Epoch [1/5], Step [600/600], Loss: 0.1289
Epoch [2/5], Step [100/600], Loss: 0.1735
Epoch [2/5], Step [200/600], Loss: 0.0641
Epoch [2/5], Step [300/600], Loss: 0.1503
Epoch [2/5], Step [400/600], Loss: 0.1623
Epoch [2/5], Step [500/600], Loss: 0.2048
Epoch [2/5], Step [600/600], Loss: 0.0652
Epoch [3/5], Step [100/600], Loss: 0.1088
Epoch [3/5], Step [200/600], Loss: 0.0448
Epoch [3/5], Step [300/600], Loss: 0.0510
Epoch [3/5], Step [400/600], Loss: 0.0711
Epoch [3/5], Step [500/600], Loss: 0.0751
Epoch [3/5], Step [600/600], Loss: 0.0356
Epoch [4/5], Step [100/600], Loss: 0.0910
Epoch [4/5], Step [200/600], Loss: 0.0647
Epoch [4/5], Step [300/600], Loss: 0.0144
Epoch [4/5], Step [400/600], Loss: 0.0308
Epoch [4/5], Step [500/600], Loss: 0.0165
Epoch [4/5], Step [600/600], Loss: